Unfortunately, there is also no way of programmatically getting at the information that USGS staff have put together for their profile pages, so I had to come up with a scraper for that information. I put this into a single function that operates against a given staff profile URL and am currently limiting the work to anything that we can't get from other sources. The interesting bits include a set of keywords, drawn from the USGS Thesaurus, that represent self-assertions of expertise. The other main section that an individual can control themselves is a big body of HTML. We might be able to work through this content with further processing, but for now, I shove that into a string and extract all of the links from it in a list. The links often include publications that are not otherwise part of the listing coming from official channels, which can be a very useful addendum to a research record.

The following code sequence grabs up our previously built inventory and runs an example to show what is returned by the scrape_profile() function.

In [1]:
import pylinkedcmd
import pickle

usgs_web = pylinkedcmd.pylinkedcmd.UsgsWeb()

In [2]:
infile = open("usgs_staff.pkl", "rb")
usgs_staff = pickle.load(infile)
infile.close()

In [3]:
usgs_staff[99]

{'name': 'Kate E Allstadt, Ph.D.',
 'profile': 'https://usgs.gov/staff-profiles/kate-e-allstadt',
 'email': 'kallstadt@usgs.gov'}

In [4]:
usgs_web.scrape_profile(usgs_staff[99]["profile"])

{'profile': 'https://usgs.gov/staff-profiles/kate-e-allstadt',
 'expertise': ['earthquakes',
  'hazards',
  'lahars',
  'landslides',
  'liquefaction'],
 'scraped_body_html': '<div class="usgs-body">\n<h2 class="h3">Biography</h2><div class="tex2jax"><p>I joined the team at the USGS Geologic Hazards Science Center in Golden, CO in June 2015 and split my time between the Landslide Hazards Program and the Earthquake Hazards Program. I focus on multidisciplinary applications of seismic and geophysical techniques to landslide and earthquake hazards. I work on a variety of topics including seismically-induced landslide and liquefaction, near-real-time earthquake products, and seismic monitoring and geophysical investigations of landslides and debris flows.</p>\n<p>As a postdoc at the Cascades Volcano Observatory, I cofounded\xa0the ongoing <a href="http://www.mshslc.org/volcanic-explorations/youth-programs/geogirls/">GeoGirls at Mount St. Helens</a> field camp designed to keep middle school